In [341]:
#imports

import requests
import time
import string
import unicodedata
import pprint

import pandas as pd

In [342]:
#read in our data
df = pd.read_csv('result_df_july18_1136am.csv')

For this specific subsample example, we need to add back S2 metadata like S2 titles and pubdates (there were left out/removed somehow when this VIAF subsample was created

In [343]:
#for this specific subsample example, we need to add back S2 info like S2 titles and pub dates

#add in the S2 titles

#get publication date to do avergae pub date work
# Load the metadata TSV file into a DataFrame
file_path = 'LitMetadataWithS2 (3).tsv'

import pandas as pd
meta = pd.read_csv(file_path, sep='\t')



In [344]:
# Ensure the authors column is of type list
meta['author'] = meta['authors'].apply(lambda x: eval(x) if isinstance(x, str) else x)

# Use the explode method to expand the authors column
meta_exploded = meta.explode('authors')

meta = meta_exploded

# Display the exploded DataFrame
# print(meta_exploded)


In [345]:
for idx, row in meta.iterrows():
    author = str(row['author'])
    author_clean = normalize_text(author)
    meta.at[idx, 'author'] = author_clean

Add back the S2 titles

In [346]:
# Initialize an empty dictionary to store the author-title mapping
author_titles = {}

# Iterate over the DataFrame rows
for index, row in meta.iterrows():
    author = str(row['author'])
    title = row['S2titles']
    
    # Use setdefault to initialize the list if the author is not already in the dictionary
    author_titles.setdefault(author, []).append(title)

# # Display the dictionary to verify
# print(author_titles)


In [347]:
import re

def normalize_text(text):
    """
    Normalize the text by converting to lowercase, preserving commas and spaces,
    and removing unnecessary special characters.
    """
    if pd.isna(text):
        return ''
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces with a single space
    text = re.sub(r'[^\w\s,]', '', text)  # Remove all characters except alphanumeric, spaces, and commas
    return text.strip()  # Trim leading and trailing spaces

title_author_dict = author_titles
# Normalize the dictionary
#normalized_dict = {normalize_text(title): normalize_text(author) for title, author in title_author_dict.items()}
#normalized_dict = {normalize_text(title): normalize_text(author) for title, author in title_author_dict.items()}

# Normalize the DataFrame
# df['normalized_S2_titlelist'] = df['S2_Titlelist'].apply(normalize_text)
df['normalized_author'] = df['author'].apply(normalize_text)

In [348]:
normalized_dict = {normalize_text(str(author)): normalize_text(str(title)) for title, author in author_titles.items()}


In [349]:
swapped_dict = {v: k for k, v in normalized_dict.items()}


In [350]:
# swapped_dict

In [351]:
df['S2_Titlelist'] = df['normalized_author'].map(swapped_dict)


In [352]:
# df

Now add back the S2 pubyears

In [353]:
# Initialize an empty dictionary to store the author-title mapping
author_years = {}

# Iterate over the DataFrame rows
for index, row in meta.iterrows():
    author = str(row['author'])
    year = row['year']
    
    # Use setdefault to initialize the list if the author is not already in the dictionary
    author_years.setdefault(author, []).append(year)

# Display the dictionary to verify
# print(author_titles)


In [354]:
# normalized_dict = {normalize_text(str(author)): normalize_text(str(title)) for title, author in title_author_dict.items()}
normalized_dict = {normalize_text(str(author)): title for title, author in author_years.items()}


In [355]:
#normalize then apply dictionary
# cleaned_dict = {key.strip("[]'"): value for key, value in normalized_dict.items()}
swapped_dict = {v: k for k, v in normalized_dict.items()}


def update_pubdates(name):
    if name in swapped_dict:
        years = swapped_dict[name]
        return years
    
df['S2_pubdates'] = df['normalized_author'].apply(update_pubdates)

In [356]:
# df['S2_pubdates'] = df['author'].map(author_years)

In [357]:
df['S2_pubdates']

0                                                   None
1                                                   None
2                                                   None
3                                                   None
4                                                   None
                             ...                        
806       1983, 1977, 1977, 1978, 1982, 1982, 2007, 1989
807       1983, 1977, 1977, 1978, 1982, 1982, 2007, 1989
808       1983, 1977, 1977, 1978, 1982, 1982, 2007, 1989
809             1994, 1971, 1976, 1987, 1986, 1988, 1993
810    2008, 2014, 2008, 2006, 2015, 2013, 2004, 2003...
Name: S2_pubdates, Length: 811, dtype: object

In [358]:
# df

Step 1: Create the Features We Need

Get all of the necessary information/entries from VIAF
(For the subsample example, this has already been done and can be skipped)

First, set up the functions to search VIAF and intialize the API

In [359]:
def get_name(name):
    params = {
        'query' : f'local.personalNames = "{name}"',
        'maximumRecords': 10,
        'startRecord' : 1,
        'sortKeys': 'holdingscount',
        'httpAccept': 'application/json'
    }

    headers={'User-Agent': user_agent}
    url = "https://viaf.org/viaf/search"

    r = requests.get(url,params=params,headers=headers, timeout=10)
    data = r.json()

    return data

In [360]:
user_agent = 'YOUR PROJECT NAME HERE'
id_column_name = "Name"

pause_between_req = 1

use_title_reconcilation = True

cache = {}

Then create a function to extract the information that we want out of the VIAF JSON returned

In [361]:
# import pandas as pd
# from fuzzywuzzy import fuzz

# birthdates_df = df

# # Function to perform search and return DataFrame with search results
# def search_author(author_name):
#     # Initialize list to store search results
#     search_results = []

#     try:
#         data = get_name(author_name)
#         records = data['searchRetrieveResponse']['records']
#         total_records = len(records)

#         for idx, record in enumerate(records):
#             record_data = record['record']['recordData']
#             birthdate = record['record']['recordData']['birthDate']
#             viaf_title_list = []
#             # Extract titles if available
#             if 'titles' in record_data:
#                 titles_data = record_data['titles']
#                 if titles_data is not None:
#                     if isinstance(titles_data['work'], list):
#                         # Extract titles from the list of works
#                         viaf_title_list.extend([work['title'] for work in titles_data['work'] if 'title' in work])
#                     else:
#                         # Extract title from a single work
#                         title = titles_data['work'].get('title')
#                         if title:
#                             viaf_title_list.append(title)

# #             # Get S2 titles for the author
# #             s2_titles = birthdates_df.loc[birthdates_df['author'] == author_name, 'S2 Titles'].iloc[0]
# #             if isinstance(s2_titles, float):
# #                 s2_titles = []

# #             # Perform fuzzy title matching
# #             title_matched = False
# #             matched_title = None
# #             for title_from_viaf in title_list:
# #                 for s2_title in s2_titles:
# #                     if fuzz.partial_ratio(s2_title, title_from_viaf) >= 70:
# #                         title_matched = True
# #                         matched_title = s2_title
# #                         break
# #                 if title_matched:
# #                     break

# #             # Append author name, title list, S2 titles, title matched, matched title, and birthdate to search results
# #             # birthdate = birthdates_df.loc[birthdates_df['author'] == author_name, 'birthdate'].iloc[0]
# #             search_results.append({'author': author_name, 'record_count': len(records),
# #                                    'record_enumerated': idx, 'title_list': title_list,
# #                                    'S2 Titles': s2_titles, 'title_matched': title_matched,
# #                                    'matched_title': matched_title, 'birthdate': birthdate})
            
#             # Append author name, title list, S2 titles, title matched, matched title, and birthdate to search results
#             # birthdate = birthdates_df.loc[birthdates_df['author'] == author_name, 'birthdate'].iloc[0]
#             title_list = df.loc[df['author'] == author_name, 'title_list'].iloc[0]

#             search_results.append({'author': author_name, 'record_count': len(records),
#                                    'record_enumerated': idx, 'viaf_title_list': viaf_title_list,
#                                    'birthdate': birthdate,'title_list':title_list})
#     except Exception as e:
#         print(f'Processing Error for {author_name}: {e}')

#     return search_results

# # Iterate through each author name in the original DataFrame and perform the search
# all_search_results = []
# row_counter = 0  # Counter to limit the number of rows processed

# for author_name in birthdates_df['author']:
#     search_results = search_author(author_name)
#     all_search_results.extend(search_results)
#     # row_counter += 1
#     # if row_counter >= 5:  # Adjust the number as needed
#     #     break

# # Convert list of dictionaries to DataFrame
# all_search_results_df = pd.DataFrame(all_search_results)
# print(all_search_results_df)


Rename some columns so they match later usage

In [362]:
result_df = df
result_df['VIAF_titlelist'] = result_df['record_enumerated_titles'] 
result_df['S2_titlelist'] = result_df['S2titles']
result_df['average_S2_pubdate'] = result_df['avg_pubdates']
result_df['VIAF_birthdate'] = result_df['standard_birthdate']
result_df['S2_Author'] = result_df['author']


df = result_df

In [363]:
df

,Unnamed: 0,index,author,birthdate,record_enumerated,title_list,record_enumerated_titles,record_count,selected_birthyear,standard_birthdate,...,common_words,notes,normalized_author,S2_Titlelist,S2_pubdates,VIAF_titlelist,S2_titlelist,average_S2_pubdate,VIAF_birthdate,S2_Author
0,0,0,NaN,1869,1,"['All men are brothers', 'All men are brothers...","['All men are brothers', 'All men are brothers...",10,NaN,1869,...,[],NaN,,NaN,None,"['All men are brothers', 'All men are brothers...",[''],NaN,1869,NaN
1,1,1,NaN,1813,2,"['Adagio, clarinet, strings, D♭ major', 'Adieu...","['Adagio, clarinet, strings, D♭ major', 'Adieu...",10,NaN,1813,...,[],NaN,,NaN,None,"['Adagio, clarinet, strings, D♭ major', 'Adieu...",[''],NaN,1813,NaN
2,2,2,NaN,1859,3,"['Adventure of Black Peter', 'Adventure of Bla...","['Adventure of Black Peter', 'Adventure of Bla...",10,NaN,1859,...,[],NaN,,NaN,None,"['Adventure of Black Peter', 'Adventure of Bla...",[''],NaN,1859,NaN
3,3,3,NaN,1866,4,"['10 letters of Sun Yat-Sen, 1914-1916', '30 J...","['10 letters of Sun Yat-Sen, 1914-1916', '30 J...",10,NaN,1866,...,[],NaN,,NaN,None,"['10 letters of Sun Yat-Sen, 1914-1916', '30 J...",[''],NaN,1866,NaN
4,4,4,NaN,1864,5,['Alle guten Wünsche zum neuen Jahre sendet I...,['Alle guten Wünsche zum neuen Jahre sendet I...,10,NaN,1864,...,[],NaN,,NaN,None,['Alle guten Wünsche zum neuen Jahre sendet I...,[''],NaN,1864,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
806,806,806,Paul Alpers,1941,1,"['1375-1620', '1620-1789', '1789-1888', '476-1...","['1375-1620', '1620-1789', '1789-1888', '476-1...",6,1932.0,1941,...,['modern'],NaN,paul alpers,"convening and convention in pastoral poetry, t...","1983, 1977, 1977, 1978, 1982, 1982, 2007, 1989","['1375-1620', '1620-1789', '1789-1888', '476-1...",['convening and convention in pastoral poetry'...,1984.375000,1941,Paul Alpers
807,807,807,Paul Alpers,1932,2,['Afterlives of Pastoral : in memorian Paul Al...,['Afterlives of Pastoral : in memorian Paul Al...,6,1932.0,1932,...,"['queene', 'poetry', 'pastoral', 'faerie', 'pa...",NaN,paul alpers,"convening and convention in pastoral poetry, t...","1983, 1977, 1977, 1978, 1982, 1982, 2007, 1989",['Afterlives of Pastoral : in memorian Paul Al...,['convening and convention in pastoral poetry'...,1984.375000,1932,Paul Alpers
808,808,808,Paul Alpers,1887,3,"['Alte deutsche Schwänke', 'Celler Flurnamenb...","['Alte deutsche Schwänke', 'Celler Flurnamenb...",6,1932.0,1887,...,[],NaN,paul alpers,"convening and convention in pastoral poetry, t...","1983, 1977, 1977, 1978, 1982, 1982, 2007, 1989","['Alte deutsche Schwänke', 'Celler Flurnamenb...",['convening and convention in pastoral poetry'...,1984.375000,1887,Paul Alpers
809,809,809,Ihab Hassan,1925,1,['Between the Eagle and the Sun : traces of Ja...,['Between the Eagle and the Sun : traces of Ja...,3,1925.0,1925,...,"['literature', 'american', 'criticism', 'postm...",NaN,ihab hassan,criticism in our clime parables of american ac...,"1994, 1971, 1976, 1987, 1986, 1988, 1993",['Between the Eagle and the Sun : traces of Ja...,['criticism in our clime: parables of american...,1985.000000,1925,Ihab Hassan


create our pubage variable

In [364]:
#create our defined function variables
import pandas as pd
from ast import literal_eval

def find_max_pubdate(pubdates):
    if isinstance(pubdates, list) and all(isinstance(pubdate, int) for pubdate in pubdates):
        return max(pubdates)
    return None

def find_min_pubdate(pubdates):
    if isinstance(pubdates, list) and all(isinstance(pubdate, int) for pubdate in pubdates):
        return min(pubdates)
    return None
    
# def find_avg_pubdate(pubdates):
#     if isinstance(pubdates, list) and all(isinstance(pubdate, int) for pubdate in pubdates):
#         return (sum(pubdates)/len(pubdates))
#     return None

def birth2maxdate(birth, pubdates):
        # Convert the string representation to an actual tuple
        pubdates_tuple = ast.literal_eval(pubdates)
        
        # Check if it's a tuple of integers
        if isinstance(pubdates_tuple, tuple) and all(isinstance(date, int) for date in pubdates_tuple):
            max_pubdate = find_max_pubdate(pubdates)
            if max_pubdate is not None:
                birth2maxdate = max_pubdate - birth
                abs_birth2maxdate = abs(birth2maxdate)
                return birth2maxdate, abs_birth2maxdate
        return None, None

def birth2mindate(birth, pubdates):
    min_pubdate = find_min_pubdate(pubdates)
    if min_pubdate is not None:
        birth2mindate = min_pubdate - birth
        abs_birth2mindate = abs(birth2mindate)
        return birth2mindate, abs_birth2mindate
    else:
        return None, None

def any_negative(birth2maxdate, birth2mindate):
    if birth2maxdate is not None and birth2mindate is not None:
        if birth2maxdate < 0 or birth2mindate < 0:
            return 1
    return 0

def title_list_len(title_list):
    return len(title_list) if isinstance(title_list, list) else 0

def author_length(author):
    return len(author)

# Master function to process each row
def process_row(row):
    birth = row['VIAF_birthdate']
    pubdates_str = row['S2_pubdates']
    author = str(row['author'])
    
    if isinstance(pubdates_str, str) and pubdates_str.strip():
        try:
            pubdates = literal_eval(pubdates_str)
        except (SyntaxError, ValueError):
            pubdates = None
    else:
        pubdates = None
    
    if pubdates is None or not isinstance(pubdates, list):
        return {
            'birth2maxdate': None,
            'abs_birth2maxdate': None,
            'birth2mindate': None,
            'abs_birth2mindate': None,
            'negative_status': None,
            'title_length': title_list_len(row['S2_Titlelist']),
            'author_length': author_length(author),
            'S2_pubdates': pubdates,
            'birthyear':birth,
            'S2 titlelist':row['S2_Titlelist'],
            'VIAF_titlelist':row['VIAF_titlelist'],
            'selected_birthyear':row['selected_birthyear'],
            'author':row['author']

        }

    birth2maxdate_value, absbirth2maxdate_value = birth2maxdate(birth, pubdates)
    birth2mindate_value, absbirth2mindate_value = birth2mindate(birth, pubdates)
    neg_status = any_negative(birth2maxdate_value, birth2mindate_value)
    title_length = title_list_len(row['S2_Titlelist'])
    author_len = author_length(row['author'])
    avg_pubdate = find_avg_pubdate(pubdates)

    return {
        'birth2maxdate': birth2maxdate_value,
        'abs_birth2maxdate': absbirth2maxdate_value,
        'birth2mindate': birth2mindate_value,
        'abs_birth2mindate': absbirth2mindate_value,
        'negative_status': neg_status,
        'title_length': title_length,
        'author_length': author_len,
        'S2_pubdates': pubdates,
        'birthyear':birth,
        'S2 titlelist':row['S2_Titlelist'],
        'VIAF_titlelist':row['VIAF_titlelist'],
        'selected_birthyear':row['selected_birthyear'],
        'author':row['author']
        


    }

# Assuming df is your DataFrame
df = result_df.apply(process_row, axis=1, result_type='expand')

# Print or use result_df as needed
# print(result_df)


In [365]:
df

,birth2maxdate,abs_birth2maxdate,birth2mindate,abs_birth2mindate,negative_status,title_length,author_length,S2_pubdates,birthyear,S2 titlelist,VIAF_titlelist,selected_birthyear,author
0,None,None,None,None,None,0,3,None,1869,NaN,"['All men are brothers', 'All men are brothers...",NaN,NaN
1,None,None,None,None,None,0,3,None,1813,NaN,"['Adagio, clarinet, strings, D♭ major', 'Adieu...",NaN,NaN
2,None,None,None,None,None,0,3,None,1859,NaN,"['Adventure of Black Peter', 'Adventure of Bla...",NaN,NaN
3,None,None,None,None,None,0,3,None,1866,NaN,"['10 letters of Sun Yat-Sen, 1914-1916', '30 J...",NaN,NaN
4,None,None,None,None,None,0,3,None,1864,NaN,['Alle guten Wünsche zum neuen Jahre sendet I...,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
806,None,None,None,None,None,0,11,"(1983, 1977, 1977, 1978, 1982, 1982, 2007, 1989)",1941,"convening and convention in pastoral poetry, t...","['1375-1620', '1620-1789', '1789-1888', '476-1...",1932.0,Paul Alpers
807,None,None,None,None,None,0,11,"(1983, 1977, 1977, 1978, 1982, 1982, 2007, 1989)",1932,"convening and convention in pastoral poetry, t...",['Afterlives of Pastoral : in memorian Paul Al...,1932.0,Paul Alpers
808,None,None,None,None,None,0,11,"(1983, 1977, 1977, 1978, 1982, 1982, 2007, 1989)",1887,"convening and convention in pastoral poetry, t...","['Alte deutsche Schwänke', 'Celler Flurnamenb...",1932.0,Paul Alpers
809,None,None,None,None,None,0,11,"(1994, 1971, 1976, 1987, 1986, 1988, 1993)",1925,criticism in our clime parables of american ac...,['Between the Eagle and the Sun : traces of Ja...,1925.0,Ihab Hassan


In [366]:
import pandas as pd
import ast


# Function to compute average publication date
def compute_avg_pubdate(pubdates):
    try:
        # Convert the string representation to an actual tuple
        pubdates_tuple = ast.literal_eval(pubdates)
        
        # Check if it's a tuple of integers
        if isinstance(pubdates_tuple, tuple) and all(isinstance(date, int) for date in pubdates_tuple):
            return sum(pubdates_tuple) / len(pubdates_tuple)
        else:
            return None
    except (ValueError, SyntaxError):
        return None

# Apply the function to each row
df['avg_pubdate'] = df['S2_pubdates'].apply(lambda x: compute_avg_pubdate(str(x)))



In [367]:
#publication_age
df['publication_age'] = df['avg_pubdate'] - df['birthyear']

In [368]:
df

,birth2maxdate,abs_birth2maxdate,birth2mindate,abs_birth2mindate,negative_status,title_length,author_length,S2_pubdates,birthyear,S2 titlelist,VIAF_titlelist,selected_birthyear,author,avg_pubdate,publication_age
0,None,None,None,None,None,0,3,None,1869,NaN,"['All men are brothers', 'All men are brothers...",NaN,NaN,NaN,NaN
1,None,None,None,None,None,0,3,None,1813,NaN,"['Adagio, clarinet, strings, D♭ major', 'Adieu...",NaN,NaN,NaN,NaN
2,None,None,None,None,None,0,3,None,1859,NaN,"['Adventure of Black Peter', 'Adventure of Bla...",NaN,NaN,NaN,NaN
3,None,None,None,None,None,0,3,None,1866,NaN,"['10 letters of Sun Yat-Sen, 1914-1916', '30 J...",NaN,NaN,NaN,NaN
4,None,None,None,None,None,0,3,None,1864,NaN,['Alle guten Wünsche zum neuen Jahre sendet I...,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
806,None,None,None,None,None,0,11,"(1983, 1977, 1977, 1978, 1982, 1982, 2007, 1989)",1941,"convening and convention in pastoral poetry, t...","['1375-1620', '1620-1789', '1789-1888', '476-1...",1932.0,Paul Alpers,1984.375000,43.375000
807,None,None,None,None,None,0,11,"(1983, 1977, 1977, 1978, 1982, 1982, 2007, 1989)",1932,"convening and convention in pastoral poetry, t...",['Afterlives of Pastoral : in memorian Paul Al...,1932.0,Paul Alpers,1984.375000,52.375000
808,None,None,None,None,None,0,11,"(1983, 1977, 1977, 1978, 1982, 1982, 2007, 1989)",1887,"convening and convention in pastoral poetry, t...","['Alte deutsche Schwänke', 'Celler Flurnamenb...",1932.0,Paul Alpers,1984.375000,97.375000
809,None,None,None,None,None,0,11,"(1994, 1971, 1976, 1987, 1986, 1988, 1993)",1925,criticism in our clime parables of american ac...,['Between the Eagle and the Sun : traces of Ja...,1925.0,Ihab Hassan,1985.000000,60.000000


In [369]:
df['publication_age'] = pd.to_numeric(df['publication_age'], errors='coerce')


Add status variable, and then convert it from string to numbers

In [370]:
df['status'] = ""

for idx, row in df.iterrows():
    if row['publication_age'] < 0:
        df.at[idx,'status'] = 'not_born'
    elif row['publication_age'] > 100:
        df.at[idx,'status'] = 'zombie'
    elif row['publication_age'] < 9:
        df.at[idx,'status'] = 'toddler'
    else:
        df.at[idx,'status'] = '0'



In [371]:
df.loc[df['status'] != '0']

,birth2maxdate,abs_birth2maxdate,birth2mindate,abs_birth2mindate,negative_status,title_length,author_length,S2_pubdates,birthyear,S2 titlelist,VIAF_titlelist,selected_birthyear,author,avg_pubdate,publication_age,status
27,None,None,None,None,None,0,11,"(1995, 1999, 2008, 1994, 1993, 1991)",1835,"social comedy in austria and germany 18901933,...","['Ballads and metrical sketches, 1860:', 'The ...",NaN,John Warren,1996.666667,161.666667,zombie
42,None,None,None,None,None,0,12,"(1960, 1953, 1982, 1988, 1968, 1952, 1986, 198...",1861,torcimany de luis de averco tratado retorico g...,['The genitalia of the group Noctuidæ ... 1909...,1915.0,Frank Pierce,1972.416667,111.416667,zombie
43,None,None,None,None,None,0,12,"(1960, 1953, 1982, 1988, 1968, 1952, 1986, 198...",1855,torcimany de luis de averco tratado retorico g...,"['Address of the president:', 'American plays ...",1915.0,Frank Pierce,1972.416667,117.416667,zombie
45,None,None,None,None,None,0,12,"(1960, 1953, 1982, 1988, 1968, 1952, 1986, 198...",1841,torcimany de luis de averco tratado retorico g...,"[""Appletons' medical dictionary; an illustrate...",1915.0,Frank Pierce,1972.416667,131.416667,zombie
47,None,None,None,None,None,0,12,"(1960, 1953, 1982, 1988, 1968, 1952, 1986, 198...",1868,torcimany de luis de averco tratado retorico g...,[],1915.0,Frank Pierce,1972.416667,104.416667,zombie
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
754,None,None,None,None,None,0,18,"(1950, 1941, 1961, 1949, 1945, 1943, 1947, 194...",1815,the four brontes the lives and works of charlo...,"['Aaron Trow', 'The American senator', 'Austra...",1906.0,Kathleen Tillotson,1946.948276,131.948276,zombie
758,None,None,None,None,None,0,13,"(2000, 2004, 2006, 2011, 2015, 2009, 2011, 200...",1871,dramaturgies of sprachkritik rainer werner fas...,['Alphabetical guide to Sinhalese folklore fro...,1968.0,David Barnett,2007.666667,136.666667,zombie
761,None,None,None,None,None,0,13,"(2000, 2004, 2006, 2011, 2015, 2009, 2011, 200...",1907,dramaturgies of sprachkritik rainer werner fas...,"['Air, flute, harp', 'Balladen Viola Klavier o...",1968.0,David Barnett,2007.666667,100.666667,zombie
769,None,None,None,None,None,0,15,"(1910, 1926, 1920, 1907, 1915, 1907, 1916, 191...",1771,"elckerlijceveryman the question of priority, s...",[],1859.0,Francis A. Wood,1913.705882,142.705882,zombie


In [372]:
df['status'] = df['status'].str.replace('zombie', '1')
df['status'] = df['status'].fillna('0')
df['status'] = df['status'].str.replace('not_born', '2')
df['status'] = df['status'].str.replace('toddler', '3')


In [373]:
# df.loc[df['status'] == '2']

Now S2 and VIAF titlelist embeddings

In [374]:
#embeddings

from sentence_transformers import SentenceTransformer

# Load the pre-trained model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Function to generate embeddings for a list of titles
def get_embeddings(titles_list):
    if isinstance(titles_list, str):
        return model.encode(titles_list)
    else:
        return model.encode('')

# Generate embeddings for each row in the DataFrame
df['S2_embeddings'] = df['S2 titlelist'].apply(get_embeddings)
df['VIAF_embeddings'] = df['VIAF_titlelist'].apply(get_embeddings)

In [375]:
df['S2_titlelist'] = df['S2 titlelist']

In [376]:
# #parse embeddings
# #fix embedding issue like we determined in earlier notebook:
# import numpy as np
# import ast

# def parse_embedding(embedding_str):
#     try:
#         # Remove leading and trailing brackets
#         # embedding_str = embedding_str.strip('[]')
        
#         # Replace newlines with spaces and split by spaces
#         # embedding_list = embedding_str.replace('\n', ' ').split()
        
#         # Convert list to numpy array
#         return np.array([float(x) for x in embedding_list], dtype=np.float32)
#     except ValueError as e:
#         print(f"Error parsing embedding: {e}")
#         return np.array([])  # return an empty array in case of error


# df['S2_embeddings_array'] = df['S2_embeddings'].apply(parse_embedding)
# df['VIAF_embeddings_array'] = df['VIAF_embeddings'].apply(parse_embedding)


#I think this step isn't needed since the embeddings WERE created within this notebook



In [377]:
df['S2_embeddings'] = df['S2_embeddings'].apply(np.mean)
df['VIAF_embeddings'] = df['VIAF_embeddings'].apply(np.mean)

In [378]:
#get overlapping words and overlapping lemmas:
# Function to lemmatize text and return a set of lemmatized words

def lemmatize_text(text):
    doc = nlp(text)
    return set(token.lemma_ for token in doc if not token.is_stop and not token.is_punct)

# Function to calculate lemma overlap and return the overlapping lemmas
def calculate_lemma_overlap(text1, text2):
    text1 = str(text1)
    text2 = str(text2)
    lemmas1 = lemmatize_text(text1)
    lemmas2 = lemmatize_text(text2)
    overlap = lemmas1.intersection(lemmas2)
    return len(overlap), overlap

# # Apply the function to compute lemma overlap for each row in the DataFrame
# df[['lemma_overlap', 'overlapping_lemmas']] = df.apply(
#     lambda row: pd.Series(calculate_lemma_overlap(row['VIAF_titlelist'], row['S2_titlelist'])), 
#     axis=1)
df[['lemma_overlap', 'overlapping_lemmas']] = df.apply(
    lambda row: pd.Series(calculate_lemma_overlap(row['VIAF_titlelist'], row['S2_titlelist'])), 
    axis=1)


In [379]:
#add overlaps

#add word overlap as a new feature
import pandas as pd
from nltk.corpus import stopwords
import nltk

# Download stopwords from NLTK if you haven't already
nltk.download('stopwords')
# Define stop words
stop_words = set(stopwords.words('english'))

# Function to create bag of words after removing stop words
def bag_of_words(title):
    words = title.lower().split()
    return set([word for word in words if word not in stop_words])

# Function to find and count word overlap
def find_word_overlap(row):
    try:
        v_list = ast.literal_eval(row['VIAF_titlelist'])
        s_list = ast.literal_eval(row['S2_titlelist'])
        if isinstance(v_list, list):

            for title in v_list:
                title = str(title)
        if isinstance(s_list, list):

            for title in s_list:
                title = str(title)
        if isinstance(v_list, list):

            v_bag = bag_of_words(v_list) 
        if isinstance(s_list, list):

            s_bag = bag_of_words(s_list) 

        overlap = v_bag & s_bag
    except:
        overlap = ''
    return pd.Series([len(overlap), list(overlap)])
  
# Apply the function to each row and create two new columns
df[['word_overlap_count', 'overlapping_words']] = df.apply(find_word_overlap, axis=1)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/beccacohen/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [380]:
#embed overlapping word meaning
import spacy
import numpy as np

# Load the spaCy model
nlp = spacy.load("en_core_web_sm")

def get_word_embeddings(words):
    embeddings = [nlp(word).vector for word in words if nlp(word).has_vector]
    if not embeddings:  # Handle case where no valid embeddings
        return np.zeros(nlp.vocab.vectors.shape[1])
    mean_embedding = np.mean(embeddings, axis=0)
    return mean_embedding


df['mean_embedding'] = df['overlapping_words'].apply(get_word_embeddings)


In [381]:
#rename to keep track of where birthdate data came from
df['VIAF_birthdate'] = df['birthyear']
df['pub_age'] = df['publication_age']

In [382]:
#store the metadata to examine later with the probabilities
original_indices = df.index
original_metadata = df[['VIAF_titlelist', 'selected_birthyear', 'author', 'S2_titlelist','status','pub_age','avg_pubdate', 'VIAF_birthdate','overlapping_words','word_overlap_count','lemma_overlap','overlapping_lemmas']].copy()




In [383]:
df['status'] = df['status'].apply(pd.to_numeric)


In [384]:
df

,birth2maxdate,abs_birth2maxdate,birth2mindate,abs_birth2mindate,negative_status,title_length,author_length,S2_pubdates,birthyear,S2 titlelist,...,S2_embeddings,VIAF_embeddings,S2_titlelist,lemma_overlap,overlapping_lemmas,word_overlap_count,overlapping_words,mean_embedding,VIAF_birthdate,pub_age
0,None,None,None,None,None,0,3,None,1869,NaN,...,0.001638,-0.001305,NaN,0,{},0,[],[],1869,NaN
1,None,None,None,None,None,0,3,None,1813,NaN,...,0.001638,-0.000812,NaN,0,{},0,[],[],1813,NaN
2,None,None,None,None,None,0,3,None,1859,NaN,...,0.001638,-0.000511,NaN,0,{},0,[],[],1859,NaN
3,None,None,None,None,None,0,3,None,1866,NaN,...,0.001638,-0.001297,NaN,0,{},0,[],[],1866,NaN
4,None,None,None,None,None,0,3,None,1864,NaN,...,0.001638,-0.001834,NaN,0,{},0,[],[],1864,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
806,None,None,None,None,None,0,11,"(1983, 1977, 1977, 1978, 1982, 1982, 2007, 1989)",1941,"convening and convention in pastoral poetry, t...",...,-0.000664,-0.001304,"convening and convention in pastoral poetry, t...",1,{modern},0,[],[],1941,43.375000
807,None,None,None,None,None,0,11,"(1983, 1977, 1977, 1978, 1982, 1982, 2007, 1989)",1932,"convening and convention in pastoral poetry, t...",...,-0.000664,-0.001322,"convening and convention in pastoral poetry, t...",3,"{poetry, queene, pastoral}",0,[],[],1932,52.375000
808,None,None,None,None,None,0,11,"(1983, 1977, 1977, 1978, 1982, 1982, 2007, 1989)",1887,"convening and convention in pastoral poetry, t...",...,-0.000664,-0.001751,"convening and convention in pastoral poetry, t...",0,{},0,[],[],1887,97.375000
809,None,None,None,None,None,0,11,"(1994, 1971, 1976, 1987, 1986, 1988, 1993)",1925,criticism in our clime parables of american ac...,...,-0.000870,-0.001676,criticism in our clime parables of american ac...,5,"{american, literature, perspective, postmodern...",0,[],[],1925,60.000000


In [385]:
df_2

,Column1,VIAF_titlelist,author,S2_titlelist,status,pub_age,avg_pubdates,standard_birthdate_viaf,overlapping_words,overlapping_lemmas,Class_0_Proba,Class_1_Proba,Class_Decision,True_Label,label_match?,Column2,Column3,new_label
0,0,"['All men are brothers', 'All men are brothers...",NaN,[''],0,NaN,NaN,1869,[],set(),0.84,0.16,0,0,True,NaN,NaN,0
1,1,"['Adagio, clarinet, strings, D♭ major', 'Adieu...",NaN,[''],0,NaN,NaN,1813,[],set(),0.82,0.18,0,0,True,NaN,NaN,0
2,2,"['Adventure of Black Peter', 'Adventure of Bla...",NaN,[''],0,NaN,NaN,1859,[],set(),0.83,0.17,0,0,True,NaN,NaN,0
3,3,"['10 letters of Sun Yat-Sen, 1914-1916', '30 J...",NaN,[''],0,NaN,NaN,1866,[],set(),0.82,0.18,0,0,True,NaN,NaN,0
4,4,['Alle guten Wünsche zum neuen Jahre sendet I...,NaN,[''],0,NaN,NaN,1864,[],set(),0.83,0.17,0,0,True,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
806,806,"['1375-1620', '1620-1789', '1789-1888', '476-1...",paul alpers,['convening and convention in pastoral poetry'...,0,43.0,1984.375000,1941,"[""'what"", 'modern', ""'the""]",{'modern'},0.31,0.69,0,0,True,NaN,NaN,0
807,807,['Afterlives of Pastoral : in memorian Paul Al...,paul alpers,['convening and convention in pastoral poetry'...,0,52.0,1984.375000,1932,"['pastoral', ""'what"", 'faerie', ""queene',"", ""'...","{'poetry', 'pastoral', 'queene'}",0.19,0.81,1,1,True,NaN,NaN,1
808,808,"['Alte deutsche Schwänke', 'Celler Flurnamenb...",paul alpers,['convening and convention in pastoral poetry'...,0,97.0,1984.375000,1887,[],set(),0.35,0.65,0,0,True,NaN,NaN,0
809,809,['Between the Eagle and the Sun : traces of Ja...,NaN,['criticism in our clime: parables of american...,0,60.0,1985.000000,1925,"['postmodern', 'american']","{'postmodern', 'literature', 'american', 'pers...",0.24,0.76,1,1,True,NaN,NaN,1


In [386]:
#in this case, I had done the true labels by hand, so let's add them in here
df_2 = pd.read_csv('new_labels_aug22.csv')
df['match?'] = df_2['new_label']

In [387]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_predict
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (precision_recall_curve, roc_auc_score,
                             confusion_matrix, ConfusionMatrixDisplay,
                             classification_report, accuracy_score)
import matplotlib.pyplot as plt


X = df.drop(columns=['author','VIAF_titlelist','S2_titlelist','overlapping_words','selected_birthyear','overlapping_lemmas','mean_embedding','matched_title?','match','match?','title_list','record_enumerated_titles','S2titles','matched_title_list','normalized_author','common_words','S2_Author','notes','S2_pubdates','S2_Titlelist','selected_birthyear'])  # Drop the label and metadata columns
y = df['match?']

# Optional: If you want to split into training and test sets first
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

KeyError: "['matched_title?', 'match', 'title_list', 'record_enumerated_titles', 'S2titles', 'matched_title_list', 'normalized_author', 'common_words', 'S2_Author', 'notes', 'S2_Titlelist'] not found in axis"

In [286]:
df[['birthdate','standard_birthdate','VIAF_birthdate']]

,birthdate,standard_birthdate,VIAF_birthdate
0,1869,1869,1869
1,1813,1813,1813
2,1859,1859,1859
3,1866,1866,1866
4,1864,1864,1864
...,...,...,...
806,1941,1941,1941
807,1932,1932,1932
808,1887,1887,1887
809,1925,1925,1925


In [287]:
# For DataFrames and Series, you can also check the data types of columns
print("\nData types in X_train:")
print(X_train.dtypes)
print("\nData types in y_train:")
print(y_train.dtype)


Data types in X_train:
Unnamed: 0.1            int64
Unnamed: 0              int64
index                   int64
birthdate               int64
record_enumerated       int64
record_count            int64
standard_birthdate      int64
S2Titles              float64
avg_pubdates          float64
pub_age               float64
status                  int64
average_S2_pubdate    float64
VIAF_birthdate          int64
birth2max_value       float64
birth2min_value       float64
title_list_len        float64
author_len            float64
avg_pubdate           float64
publication_age       float64
S2_embeddings         float32
VIAF_embeddings       float32
word_overlap_count      int64
lemma_overlap           int64
dtype: object

Data types in y_train:
int64


In [288]:
# For DataFrames and Series, you can also check the data types of columns
print("\nData types in X:")
print(X.dtypes)
print("\nData types in y:")
print(y.dtype)


Data types in X:
Unnamed: 0.1            int64
Unnamed: 0              int64
index                   int64
birthdate               int64
record_enumerated       int64
record_count            int64
standard_birthdate      int64
S2Titles              float64
avg_pubdates          float64
pub_age               float64
status                  int64
average_S2_pubdate    float64
VIAF_birthdate          int64
birth2max_value       float64
birth2min_value       float64
title_list_len        float64
author_len            float64
avg_pubdate           float64
publication_age       float64
S2_embeddings         float32
VIAF_embeddings       float32
word_overlap_count      int64
lemma_overlap           int64
dtype: object

Data types in y:
int64


In [392]:
# Initialize the Random Forest model
model = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced_subsample')

# Perform cross-validation
# model = RandomForestClassifier()


In [393]:
y_pred_proba = cross_val_predict(model, X, y, cv=5, method='predict_proba')

# Store predictions with original indices
predictions_df = pd.DataFrame(y_pred_proba, index=original_indices, columns=['Class_0_Proba', 'Class_1_Proba'])

# Merge the metadata back
results_df = pd.concat([original_metadata, predictions_df], axis=1)
results_df['True_Label'] = y.loc[results_df.index]

# Now you have a DataFrame with metadata and predictions
print(results_df.head())

                                      VIAF_titlelist selected_birthyear  \
0  ['All men are brothers', 'All men are brothers...                NaN   
1  ['Adagio, clarinet, strings, D♭ major', 'Adieu...                NaN   
2  ['Adventure of Black Peter', 'Adventure of Bla...                NaN   
3  ['10 letters of Sun Yat-Sen, 1914-1916', '30 J...                NaN   
4  ['Alle guten Wünsche zum neuen Jahre sendet I...                NaN   

  author S2_titlelist status  pub_age  avg_pubdate  VIAF_birthdate  \
0    NaN          NaN      0      NaN          NaN            1869   
1    NaN          NaN      0      NaN          NaN            1813   
2    NaN          NaN      0      NaN          NaN            1859   
3    NaN          NaN      0      NaN          NaN            1866   
4    NaN          NaN      0      NaN          NaN            1864   

  overlapping_words  word_overlap_count  lemma_overlap overlapping_lemmas  \
0                []                   0            

In [394]:
# y_pred_proba = cross_val_predict(model, X_train, y_train, cv=5, method='predict_proba')
# 
# # Store predictions with original indices
# predictions_df = pd.DataFrame(y_pred_proba, index=original_indices, columns=['Class_0_Proba', 'Class_1_Proba'])
# 
# # Merge the metadata back
# results_df = pd.concat([original_metadata, predictions_df], axis=1)
# results_df['True_Label'] = y.loc[results_df.index]
# 
# # Now you have a DataFrame with metadata and predictions
# print(results_df.head())

In [395]:
# #including the testing and training sets
# 
# predictions_df = pd.DataFrame(y_pred_proba, index=X_train.index, columns=['Class_0_Proba', 'Class_1_Proba'])
# 
# # Merge the metadata with predictions using original indices
# results_df = pd.concat([original_metadata.loc[X_train.index], predictions_df], axis=1)
# 
# # Add the true labels to the DataFrame
# results_df['True_Label'] = y_train
# 
# # Display the first few rows of the DataFrame
# print(results_df.head())


In [396]:
# from sklearn.metrics import classification_report
# 
# # Convert predicted probabilities to class labels (0 or 1 based on threshold 0.5)
# y_pred = (y_pred_proba[:, 1] >= 0.5).astype(int)
# 
# 
# # Generate classification report based on the true labels (y_train) and predicted labels (y_pred)
# report = classification_report(y_train, y_pred)
# 
# # Print the classification report
# print(report)

In [397]:
#all data included

y_pred_proba = cross_val_predict(model, X, y, cv=5, method='predict_proba')

# Store predictions with original indices
predictions_df = pd.DataFrame(y_pred_proba, index=original_indices, columns=['Class_0_Proba', 'Class_1_Proba'])

# Merge the metadata back
results_df = pd.concat([original_metadata, predictions_df], axis=1)
results_df['True_Label'] = y.loc[results_df.index]

# Now you have a DataFrame with metadata and predictions
print(results_df.head())

                                      VIAF_titlelist selected_birthyear  \
0  ['All men are brothers', 'All men are brothers...                NaN   
1  ['Adagio, clarinet, strings, D♭ major', 'Adieu...                NaN   
2  ['Adventure of Black Peter', 'Adventure of Bla...                NaN   
3  ['10 letters of Sun Yat-Sen, 1914-1916', '30 J...                NaN   
4  ['Alle guten Wünsche zum neuen Jahre sendet I...                NaN   

  author S2_titlelist status  pub_age  avg_pubdate  VIAF_birthdate  \
0    NaN          NaN      0      NaN          NaN            1869   
1    NaN          NaN      0      NaN          NaN            1813   
2    NaN          NaN      0      NaN          NaN            1859   
3    NaN          NaN      0      NaN          NaN            1866   
4    NaN          NaN      0      NaN          NaN            1864   

  overlapping_words  word_overlap_count  lemma_overlap overlapping_lemmas  \
0                []                   0            

In [400]:
from sklearn.metrics import classification_report

# Convert predicted probabilities to class labels (0 or 1 based on threshold 0.5)
# y_pred = (y_pred_proba[:, 1] >= 0.5).astype(int)

y_pred = (y_pred_proba[:, 1] >= 0.8).astype(int)


# Generate classification report based on the true labels (y_train) and predicted labels (y_pred)
report = classification_report(y, y_pred)

# Print the classification report
print(report)

              precision    recall  f1-score   support

           0       0.72      0.93      0.81       532
           1       0.69      0.30      0.41       279

    accuracy                           0.71       811
   macro avg       0.70      0.61      0.61       811
weighted avg       0.71      0.71      0.67       811



In [401]:
y_pred = (y_pred_proba[:, 1] >= 0.5).astype(int)


# Generate classification report based on the true labels (y_train) and predicted labels (y_pred)
report = classification_report(y, y_pred)

# Print the classification report
print(report)

              precision    recall  f1-score   support

           0       0.79      0.77      0.78       532
           1       0.58      0.61      0.60       279

    accuracy                           0.72       811
   macro avg       0.69      0.69      0.69       811
weighted avg       0.72      0.72      0.72       811



In [404]:
y_pred = (y_pred_proba[:, 1] >= 0.7).astype(int)


# Generate classification report based on the true labels (y_train) and predicted labels (y_pred)
report = classification_report(y, y_pred)

# Print the classification report
print(report)

              precision    recall  f1-score   support

           0       0.74      0.87      0.80       532
           1       0.63      0.42      0.51       279

    accuracy                           0.72       811
   macro avg       0.69      0.65      0.65       811
weighted avg       0.71      0.72      0.70       811



In [403]:
# y_pred = (y_pred_proba[:, 1] >= 0.9).astype(int)
# 
# 
# # Generate classification report based on the true labels (y_train) and predicted labels (y_pred)
# report = classification_report(y, y_pred)
# 
# # Print the classification report
# print(report)

              precision    recall  f1-score   support

           0       0.67      0.98      0.80       532
           1       0.76      0.09      0.17       279

    accuracy                           0.68       811
   macro avg       0.72      0.54      0.48       811
weighted avg       0.71      0.68      0.58       811



In [399]:
#results
# 
# from sklearn.metrics import confusion_matrix
# from sklearn.metrics import classification_report
# confusion_matrix = confusion_matrix(y,y_pred)
# print(confusion_matrix) #gives true pos, false neg etc
# 
# classification_report = classification_report(y, y_pred) #gives precision, f1_score etc
# print(classification_report)

In [405]:
#save the selected model to a pickle file
# Save the trained model to a pickle file
with open('viaf_classifier_sept23.pkl', 'wb') as file:
    pickle.dump(model, file)